add data

In [62]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=4000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

1654784/1641221 [==============================] - 0s 0us/step


In [63]:
vocabulary = list()
for text in x_train:
  tokens = text.split()
  vocabulary.extend(tokens)

vocabulary = set(vocabulary)
print(len(vocabulary))

3998


In [69]:
from tqdm import tqdm

x_train_binary = list()
x_test_binary = list()

for text in tqdm(x_train):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_train_binary.append(binary_vector)

x_train_binary = np.array(x_train_binary)

for text in tqdm(x_test):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_test_binary.append(binary_vector)

x_test_binary = np.array(x_test_binary)

100%|██████████| 25000/25000 [07:10<00:00, 58.02it/s]


In [73]:
print(x_test_binary[2])
print(y_test[2])

[0 0 0 ... 0 0 0]
1


Exaple for binarytree library

In [59]:
from binarytree import Node
root = Node(3)
root.left = Node(6)
root.right = Node(8)
 
# Getting binary tree
print('Binary tree :', root)
 
# Getting list of nodes
print('List of nodes :', list(root))
 
# Getting inorder of nodes
print('Inorder of nodes :', root.inorder)
 
# Checking tree properties
print('Size of tree :', root.size)
print('Height of tree :', root.height)
 
# Get all properties at once
print('Properties of tree : \n', root.properties)

Binary tree : 
  3
 / \
6   8

List of nodes : [Node(3), Node(6), Node(8)]
Inorder of nodes : [Node(6), Node(3), Node(8)]
Size of tree : 3
Height of tree : 1
Properties of tree : 
 {'height': 1, 'size': 3, 'is_max_heap': False, 'is_min_heap': True, 'is_perfect': True, 'is_strict': True, 'is_complete': True, 'leaf_count': 2, 'min_node_value': 3, 'max_node_value': 8, 'min_leaf_depth': 1, 'max_leaf_depth': 1, 'is_balanced': True, 'is_bst': False, 'is_symmetric': False}


Calculate entropy

In [165]:
import math
from math import log2 

def calculate_entropy_total_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  if word_position == -1:
    word_position = return_word(word) # the word column

  for element in binary_array:
      pos = element[word_position]
      count += 1
      if(pos == c):
        first += 1
      else:
        second += 1
  return -(first/count) * log2((first/count) - (second/count) * log2((second/count)))

def calculate_entropy_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth

  if word_position == -1:
    word_position = return_word(word) # the word column

  for element in binary_array:
      count += 1
      if(element == c):
        first += 1
      else:
        second += 1
  return -(first/count) * log2((first/count) - (second/count) * log2((second/count)))


def calculate_entropy_total(binary_array, c):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  for element in binary_array:
      for pos in element:
        count += 1
        if(pos == c):
          first += 1
        else:
          second += 1
  return -(first/count) * log2((first/count) - (second/count) * log2((second/count)))


  

  

0.1133869708654645


return the position of a word

In [120]:
# -1 = has not found the word
def return_word(word):
  count = 0
  for element in vocabulary:
    if word == element:
      return count
    else:
      count += 1
  
  return -1

-1


calculate probability

In [146]:
def calculate_probability(array, c):
  count = 0
  for element in array:
    if(element == c):
      count += 1

  return c / len(array) * 100 

calculate sum

In [166]:
def calculate_sum(binary_array, c, word):
  sum = 0
  word_position = return_word(word)
  for review in binary_array:
   sum += calculate_probability(review, c) * calculate_entropy_word(review,c , word,  word_position)
  return sum



print(calculate_sum(x_train_binary, 1, "each"))
    

67.74524936264618


calculate IG

In [ ]:
def calculate_IG(binary_array, c, word):
  return calculate_entropy_total(binary_array, c) - calculate_sum(binary_array, c, word)

calculate_IG(x_train_binary, 1, "love")